# Import Model

In [22]:
from python_scripts.pretrain_model import PretrainedEncoderRegressor
import torch.nn as nn

name = 'AE_Center_noaug'

checkpoint_path = f"AE_model/128/{name}/best.pt"

# 1) 实例化（会自动加载并冻结 encoder）
model = PretrainedEncoderRegressor(
    ae_checkpoint=checkpoint_path,
    ae_type="center",
    tile_dim=128,
    center_dim=128,
    neighbor_dim=128,
    output_dim=35
)

# 2) monkey‐patch 一个新的 head
model.decoder  = nn.Sequential(
    nn.Linear(128+128+128, 256),
    nn.LeakyReLU(0.01),
    nn.Dropout(0.1),
    nn.Linear(256, 128),
    nn.LeakyReLU(0.01),
    nn.Dropout(0.1),
    nn.Linear(128, 64),
    nn.LeakyReLU(0.01),
    nn.Dropout(0.1),
    nn.Linear(64, 35)
    
)

import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.act1  = nn.SiLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.shortcut = None
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        self.act2 = nn.SiLU()

    def forward(self, x):
        identity = x
        out = self.act1(self.bn1(self.conv1(x)))
        out = self.act2(self.bn2(self.conv2(out)))
        if self.shortcut is not None:
            identity = self.shortcut(x)
        return out + identity



class DeepTileEncoder(nn.Module):
    """加深的 Tile 分支：全局信息，多尺度池化 + 三层 MLP"""
    def __init__(self, out_dim, in_channels=3, negative_slope=0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 78→39
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 39→19
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128),
            nn.MaxPool2d(2)  # 19→9
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256),
            ResidualBlock(256, 256)
        )  # 保持 9×9

        # 多尺度池化
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # [B,256,1,1]
        self.mid_pool    = nn.AdaptiveAvgPool2d((3, 3))  # [B,256,3,3]

        total_dim = 256*1*1 + 256*3*3
        # 三层 MLP：total_dim → 2*out_dim → out_dim → out_dim
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*4),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*4, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x: [B,256,9,9]
        g = self.global_pool(x).contiguous().reshape(x.size(0), -1)  # [B,256]
        m = self.mid_pool(x).contiguous().reshape(x.size(0), -1)     # [B,256*3*3]

        return self.fc(torch.cat([g, m], dim=1))


class SubtileEncoder(nn.Module):
    """多尺度 Subtile 分支：局部信息 + 两层 MLP"""
    def __init__(self, out_dim, in_channels=3, negative_slope=0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 26→13
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 13→6
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128)
        )  # 保持 6×6

        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.mid_pool    = nn.AdaptiveAvgPool2d((2,2))
        self.large_pool    = nn.AdaptiveAvgPool2d((3,3))

        total_dim = 128*1*1 + 128*2*2 + 128*3*3
        # 两层 MLP：total_dim → out_dim*2 → out_dim
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        B, N, C, H, W = x.shape
        x = x.contiguous().reshape(B*N, C, H, W)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        # g,m: [B*N, feat]
        g = self.global_pool(x).contiguous().reshape(B, N, -1)
        m = self.mid_pool(x).contiguous().reshape(B, N, -1)
        l = self.large_pool(x).contiguous().reshape(B, N, -1)

        # 合并 N 张 subtiles，再 FC
        feat = torch.cat([g, m, l], dim=2).mean(dim=1).contiguous()  # [B, total_dim]
        return self.fc(feat)
class CenterSubtileEncoder(nn.Module):
    """專門處理中心 subtile 的 Encoder"""
    def __init__(self, out_dim, in_channels=3, negative_slope= 0.01):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.MaxPool2d(2)  # 26→13
        )
        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64),
            ResidualBlock(64, 64),
            nn.MaxPool2d(2)  # 13→6
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128),
            ResidualBlock(128, 128)
        )  # 6×6

        # 多尺度池化
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.mid_pool    = nn.AdaptiveAvgPool2d((2,2))
        self.large_pool    = nn.AdaptiveAvgPool2d((3,3))

        total_dim = 128*1*1 + 128*2*2 + 128*3*3
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.Linear(total_dim, out_dim*2),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(out_dim*2, out_dim),
            nn.LeakyReLU(negative_slope),
        )

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        g = self.global_pool(x).contiguous().reshape(x.size(0), -1)
        m = self.mid_pool(x).contiguous().reshape(x.size(0), -1)
        l = self.large_pool(x).contiguous().reshape(x.size(0), -1)

        return self.fc(torch.cat([g, m, l], dim=1)).contiguous()



class VisionMLP_MultiTask(nn.Module):
    """整體多任務模型：融合 tile + subtile + center，使用動態權重融合"""
    def __init__(self, tile_dim=128, subtile_dim=64, output_dim=35, negative_slope=0.01):
        super().__init__()
        self.encoder_tile    = DeepTileEncoder(tile_dim)
        self.encoder_subtile = SubtileEncoder(subtile_dim)
        self.encoder_center  = CenterSubtileEncoder(subtile_dim)

        # 融合層：輸入三個分支的 concat，輸出三個 gate
        self.gate_fc = nn.Sequential(
            nn.Linear(tile_dim + subtile_dim + subtile_dim, 64),
            nn.LeakyReLU(negative_slope),
            nn.Linear(64, 3),  # 對 tile, subtile, center 分支輸出 gate
            nn.Softmax(dim=1)  # 轉成權重
        )

        # 輸出 decoder：輸入為 tile_dim (因為融合後只剩一個 vector)
        self.decoder = nn.Sequential(
            nn.Linear(tile_dim, 256),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(0.1),
            nn.Linear(64, output_dim),
        )

    def forward(self, tile, subtiles):
        tile = tile.contiguous()
        subtiles = subtiles.contiguous()
        center = subtiles[:, 4]

        f_tile = self.encoder_tile(tile)         # [B, tile_dim]
        f_sub  = self.encoder_subtile(subtiles)  # [B, subtile_dim]
        f_center = self.encoder_center(center)   # [B, subtile_dim]

        # 拼接三個分支做 gating
        features_cat = torch.cat([f_tile, f_sub, f_center], dim=1)  # [B, tile+sub+center]
        gates = self.gate_fc(features_cat)  # [B, 3]

        # 對三個分支做 weighted sum
        f_fused = (
            gates[:, 0:1] * f_tile + 
            gates[:, 1:2] * f_sub + 
            gates[:, 2:3] * f_center
        )  # [B, tile_dim]（注意：需保證 f_tile == f_sub == f_center 的維度）

        return self.decoder(f_fused)
    
# class VisionMLP_MultiTask(nn.Module):
#     """整體多任務模型：融合 tile + subtile + center + position 特徵"""
#     def __init__(self, tile_dim=128, subtile_dim=128, output_dim=35, negative_slope=0.01):
#         super().__init__()
#         self.encoder_tile    = DeepTileEncoder(tile_dim)
#         self.encoder_subtile = SubtileEncoder(subtile_dim)
#         self.encoder_center  = CenterSubtileEncoder(subtile_dim)

#         self.feature_dim = tile_dim + subtile_dim + subtile_dim # +2 for position(x,y)

#         self.decoder = nn.Sequential(
#             nn.Linear(self.feature_dim, 256),
#             nn.LeakyReLU(negative_slope),
#             nn.Dropout(0.1),
#             nn.Linear(256, 128),
#             nn.LeakyReLU(negative_slope),
#             nn.Dropout(0.1),
#             nn.Linear(128, 64),
#             nn.LeakyReLU(negative_slope),
#             nn.Dropout(0.1),
#             nn.Linear(64, output_dim),
#         )

#     def forward(self, tile, subtiles):
#         tile = tile.contiguous()
#         subtiles = subtiles.contiguous()
#         center = subtiles[:, 4]

#         f_tile = self.encoder_tile(tile)         # [B, tile_dim]
#         f_sub  = self.encoder_subtile(subtiles)  # [B, subtile_dim]
#         f_center = self.encoder_center(center)   # [B, subtile_dim]

#         # 拼接特徵向量與座標
#         features_cat = torch.cat([f_tile, f_sub, f_center], dim=1)  # [B, tile+sub+center+2]

#         return self.decoder(features_cat)


# 用法示例
model = VisionMLP_MultiTask(tile_dim=128, subtile_dim=128, output_dim=35)


# —— 5) 确保只有 decoder 可训练 ——  
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable / total params = {trainable:,} / {total:,}")


device   = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = model.to(device)
model.load_state_dict(torch.load('output_folder/rank-spot/realign/whole_worflow/s_m_l/filtered_directly_rank/k-fold_mix/realign_all/Macenko_masked/results/model_epoch022.pt', map_location="cpu"))
model.to(device).eval()

/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/pretrain_model.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ae.load_state_dict(torch.load(ae_checkpoint, map_

Trainable / total params = 6,639,142 / 6,639,142


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_79027/3220337273.py:319: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('output_fo

VisionMLP_MultiTask(
  (encoder_tile): DeepTileEncoder(
    (layer0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        

## Load Model

# Import training data

## Same in multiple .pt

In [23]:
import os
import torch
import random
import inspect
from python_scripts.import_data import load_all_tile_data

# 用法範例
#folder = "dataset/spot-rank/version-3/only_tile_sub/original_train"
folder = "dataset/spot-rank/filtered_directly_rank/masked/realign/Macenko_masked/filtered/train_data/"

grouped_data = load_all_tile_data( 
        folder_path=folder,
        model=model,
        fraction=1,
        shuffle=False
    )

    # grouped_data 現在只會有 model.forward() 需要的 key，
    # 像 ['tile','subtiles','neighbors','norm_coord','node_feat','adj_list','edge_feat','label','source_idx']
print("Loaded keys:", grouped_data.keys())
print("Samples:", len(next(iter(grouped_data.values()))))




/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/import_data.py:251: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load(fpath, map_location='cpu')


Loaded keys: dict_keys(['position', 'source_idx', 'tile', 'label', 'slide_idx', 'subtiles'])
Samples: 8348


In [24]:
from python_scripts.import_data import importDataset

import torch
from torch.utils.data import Dataset
import inspect
import numpy as np


full_dataset = importDataset(grouped_data, model,
                             image_keys=['tile','subtiles'],
                             transform=lambda x: x)

full_dataset.check_item()

🔍 Checking dataset sample: 0
📏 tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: 0.157, max: 1.000, mean: 0.680, std: 0.142
📏 subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32 | min: 0.157, max: 1.000, mean: 0.680, std: 0.142
📏 label shape: torch.Size([35]) | dtype: torch.float32 | min: 1.000, max: 35.000, mean: 18.000, std: 10.247
--- label head (前 5 個元素):
tensor([12., 24., 18.,  6., 30.])
📏 source_idx shape: torch.Size([]) | dtype: torch.int64 | min: 0.000, max: 0.000, mean: 0.000, std: nan
--- source_idx 資料為純量: tensor(0)
📏 position shape: torch.Size([2]) | dtype: torch.float32 | min: 0.171, max: 0.632, mean: 0.401, std: 0.326
--- position head (前 5 個元素):
tensor([0.6318, 0.1707])
✅ All checks passed!


/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/import_data.py:201: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1744320376245/work/aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = tensor_float.std().item()


In [99]:

from python_scripts.image_features import  *
from python_scripts.prediction_features import  *
import numpy as np
from torch.utils.data import DataLoader

# === Main Function with Names ===
def generate_meta_features(dataset, model_for_recon, device, ae_type, oof_preds = None):
    """
    Generate meta-features and corresponding names.

    Returns
    -------
    features : np.ndarray, shape (n_samples, n_features)
    names    : list of str, length n_features
    """

    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    # 1) 收集所有 (feats, names) 到同一个 outputs 列表
    outputs = []

    # AE reconstruction loss
    feats, names = compute_ae_reconstruction_loss(model_for_recon, loader, device, ae_type)
    feats = feats[:, None]
    outputs.append((feats, names))

    # AE embeddings
    feats, names = compute_ae_embeddings(loader, model_for_recon, device)
    outputs.append((feats, names))

    # Latent stats
    latent_feats = outputs[1][0]
    feats, names = compute_latent_stats(latent_feats)
    outputs.append((feats, names))

    # RGB stats

    feats, names = compute_all_subtiles_rgb_stats(dataset)
    outputs.append((feats, names))
    feats, names = compute_subtiles_except_center_rgb_stats(dataset)
    outputs.append((feats, names))
    feats, names = compute_tile_rgb_stats(dataset)
    outputs.append((feats, names))
    feats, names = compute_subtile_contrast_stats(dataset)
    outputs.append((feats, names))

    # Texture & pattern features
    feats, names = compute_wavelet_stats(dataset)
    outputs.append((feats, names))
    feats, names = compute_sobel_stats(dataset)
    outputs.append((feats, names))

    # Color & distribution features
    feats, names = compute_hsv_stats(dataset)
    outputs.append((feats, names))
    feats, names = compute_color_moments(dataset)
    outputs.append((feats, names))

    # H&E stain features
    feats, names = compute_he_stats(dataset)
    outputs.append((feats, names))

    # Sliding-window std stats
    feats, names = compute_sliding_window_stats(dataset)
    outputs.append((feats, names))

    if oof_preds is not None:
        feats, names = compute_entropy(oof_preds)
        outputs.append((feats, names))
        feats, names = compute_top2_diff(oof_preds)
        outputs.append((feats, names))        
        feats, names = compute_pairwise_diff(oof_preds)
        outputs.append((feats, names))
        feats, names = compute_dispersion(oof_preds)
        outputs.append((feats, names))
        
    # 2) unzip 成 feat_list 和 name_seq
    feat_list, name_seq = zip(*outputs)

    # 3) 逐块校验 feats 列数与 names 长度
    for feats, names_block in zip(feat_list, name_seq):
        ncols = feats.shape[1] if feats.ndim == 2 else 1
        if ncols != len(names_block):
            raise ValueError(
                f"Mismatch: got {ncols} columns but {len(names_block)} names "
                f"in block '{names_block[0].split('_')[0]}'"
            )
        print(
            f"{names_block[0].split('_')[0]:12s} -> cols: {ncols:4d}, names: {len(names_block):4d} OK"
        )

    # 4) 扁平化 names 并拼接 features
    name_list = [nm for block in name_seq for nm in block]
    features = np.concatenate(feat_list, axis=1)
    print(f"✅ Generated meta-features with shape: {features.shape}")

    return features, name_list


In [100]:
import os
import numpy as np
import joblib
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from scipy.stats import rankdata
from python_scripts.import_data import importDataset
from python_scripts.operate_model import predict
from lightgbm import early_stopping, log_evaluation
import h5py
import pandas as pd
from python_scripts.pretrain_model import PretrainedEncoderRegressor
# ---------------- Settings ----------------

version = 'version3'
save_folder = f"only_lightgbm/{version}"  # 修改為你想要的資料夾名稱
if not os.path.exists(save_folder):   
    os.makedirs(save_folder)
n_samples  = len(full_dataset)
C          = 35
BATCH_SIZE = 64
start_fold = 0

tile_dim = 128
center_dim = 128
neighbor_dim = 128
fusion_dim = tile_dim + center_dim + neighbor_dim

pretrained_ae_name = 'AE_Center_noaug'
pretrained_ae_path = f"AE_model/128/{pretrained_ae_name}/best.pt"
ae_type = 'center'

# Ground truth label (全 dataset)
y_true = np.vstack([ full_dataset[i]['label'].cpu().numpy() for i in range(n_samples) ])

# Build CV splitter (must match first stage splits)
logo = LeaveOneGroupOut()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

lgb_base = lgb.LGBMRegressor(
    objective='l2',
    metric='rmse',
    learning_rate=0.01,
    n_estimators=20000,
    max_depth=15,
    num_leaves=127,
    colsample_bytree=0.7619407413363416,
    subsample=0.8,
    subsample_freq=1,
    min_data_in_leaf=20,
    reg_alpha=0.7480401395491829,
    reg_lambda=0.2589860348178542,
    verbosity=-1
    )

slide_idx = np.array(grouped_data['slide_idx'])   # shape (N,)
recon_model = PretrainedEncoderRegressor(
        ae_checkpoint=pretrained_ae_path,
        ae_type=ae_type,
        tile_dim=tile_dim,
        center_dim=center_dim,
        neighbor_dim=neighbor_dim,
        output_dim=C,
        mode='reconstruction'
    ).to(device)
all_meta, name = generate_meta_features(
    full_dataset,
    model_for_recon = recon_model,
    device = device,
    ae_type = ae_type,
)  # shape (N,307) or whatever

/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/pretrain_model.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ae.load_state_dict(torch.load(ae_checkpoint, map_

ae-recon-loss -> cols:    1, names:    1 OK
ae           -> cols:  384, names:  384 OK
latent       -> cols:    4, names:    4 OK
subtile      -> cols:  108, names:  108 OK
exsubtiles   -> cols:   12, names:   12 OK
tile         -> cols:   12, names:   12 OK
contrast     -> cols:    6, names:    6 OK
wavelet-tile -> cols:  280, names:  280 OK
sobel-tile   -> cols:   40, names:   40 OK
hsv-tile     -> cols:  120, names:  120 OK
color-tile   -> cols:  120, names:  120 OK
he-tile      -> cols:   80, names:   80 OK
locstd       -> cols:  432, names:  432 OK
✅ Generated meta-features with shape: (8348, 1599)


In [102]:

# 5a) Optional: split a small val‐set for early stopping
X_tr, X_val, y_tr, y_val = train_test_split(
    all_meta, y_true, test_size=0.2, random_state=42
)

meta_model = MultiOutputRegressor(lgb_base)
meta_model.estimators_ = []
for i in range(C):
    print(f"Training meta‐model target {i} …")
    m = lgb.LGBMRegressor(**lgb_base.get_params())
    m.fit(
        X_tr, y_tr[:, i],
        eval_set=[(X_val, y_val[:, i])],
        callbacks=[early_stopping(stopping_rounds=200), log_evaluation(period=100)]
    )
    meta_model.estimators_.append(m)

save_folder = f"only_lightgbm/{version}"  # 修改為你想要的資料夾名稱
if not os.path.exists(save_folder):   
    os.makedirs(save_folder)
# 6) Save the single meta‐model
joblib.dump(meta_model, f'{save_folder}/meta_model_single.pkl')
print("✅ Saved entire‐dataset meta‐model → meta_model_single.pkl")

Training meta‐model target 0 …


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 5.9541
[200]	valid_0's rmse: 5.52642
[300]	valid_0's rmse: 5.42383
[400]	valid_0's rmse: 5.3908
[500]	valid_0's rmse: 5.37594
[600]	valid_0's rmse: 5.37204
[700]	valid_0's rmse: 5.37
[800]	valid_0's rmse: 5.36744
[900]	valid_0's rmse: 5.36765
[1000]	valid_0's rmse: 5.36708
[1100]	valid_0's rmse: 5.36786
Early stopping, best iteration is:
[933]	valid_0's rmse: 5.3668
Training meta‐model target 1 …


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.68238
[200]	valid_0's rmse: 2.46078
[300]	valid_0's rmse: 2.40458
[400]	valid_0's rmse: 2.39047
[500]	valid_0's rmse: 2.38375
[600]	valid_0's rmse: 2.38134
[700]	valid_0's rmse: 2.37876
[800]	valid_0's rmse: 2.3764
[900]	valid_0's rmse: 2.37535
[1000]	valid_0's rmse: 2.37459
[1100]	valid_0's rmse: 2.37411
[1200]	valid_0's rmse: 2.37339
[1300]	valid_0's rmse: 2.37295
[1400]	valid_0's rmse: 2.37268
[1500]	valid_0's rmse: 2.37255
[1600]	valid_0's rmse: 2.37241
[1700]	valid_0's rmse: 2.37246
[1800]	valid_0's rmse: 2.37237
[1900]	valid_0's rmse: 2.37243
Early stopping, best iteration is:
[1789]	valid_0's rmse: 2.37233
Training meta‐model target 2 …


...

Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 7.48294
[200]	valid_0's rmse: 7.42035
[300]	valid_0's rmse: 7.41268
[400]	valid_0's rmse: 7.41583
Early stopping, best iteration is:
[295]	valid_0's rmse: 7.41143
Training meta‐model target 33 …


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 5.73004
[200]	valid_0's rmse: 5.53418
[300]	valid_0's rmse: 5.48538
[400]	valid_0's rmse: 5.47701
[500]	valid_0's rmse: 5.47599
[600]	valid_0's rmse: 5.47595
[700]	valid_0's rmse: 5.47676
Early stopping, best iteration is:
[516]	valid_0's rmse: 5.47389
Training meta‐model target 34 …


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 2.94864
[200]	valid_0's rmse: 2.86409
[300]	valid_0's rmse: 2.84439
[400]	valid_0's rmse: 2.83818
[500]	valid_0's rmse: 2.83595
[600]	valid_0's rmse: 2.83355
[700]	valid_0's rmse: 2.83247
[800]	valid_0's rmse: 2.83121
[900]	valid_0's rmse: 2.83125
[1000]	valid_0's rmse: 2.83143
Early stopping, best iteration is:
[807]	valid_0's rmse: 2.83109
✅ Saved entire‐dataset meta‐model → meta_model_single.pkl


In [ ]:
import torch
from python_scripts.import_data import load_node_feature_data


image_keys = [ 'tile', 'subtiles']

model = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)

# 用法示例
from python_scripts.import_data import importDataset
# 假设你的 model 已经定义好并实例化为 `model`
test_dataset = load_node_feature_data("dataset/spot-rank/filtered_directly_rank/masked/test/Macenko/test_dataset.pt", model)
test_dataset = importDataset(
        data_dict=test_dataset,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
    )



/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/import_data.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  raw = torch.load(pt_path, map_location="cpu")
Except

⚠️ 從 '<class 'list'>' 推斷樣本數量: 2088
Model forward signature: (tile, subtiles)


In [103]:
import joblib

meta_test, test_name = generate_meta_features(
    dataset         = test_dataset,
    model_for_recon = recon_model,
    device          = device,
    ae_type         = ae_type,
)

# 3) 直接载入并用 single‐fold 的 meta_model 预测
meta_model = joblib.load(f'{save_folder}/meta_model_single.pkl')
final_preds = meta_model.predict(meta_test)

# 4) 写出 submission
import h5py, pandas as pd
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
sub = pd.DataFrame(final_preds, columns=[f"C{i+1}" for i in range(C)])
sub.insert(0, 'ID', test_spot_ids.index)
sub.to_csv(f'{save_folder}/submission_stacked.csv', index=False)
print("✅ Saved submission_stacked.csv")


Computing AE recon loss: 100%|██████████| 33/33 [00:08<00:00,  3.72it/s]


ae-recon-loss -> cols:    1, names:    1 OK
ae           -> cols:  384, names:  384 OK
latent       -> cols:    4, names:    4 OK
subtile      -> cols:  108, names:  108 OK
exsubtiles   -> cols:   12, names:   12 OK
tile         -> cols:   12, names:   12 OK
contrast     -> cols:    6, names:    6 OK
wavelet-tile -> cols:  280, names:  280 OK
sobel-tile   -> cols:   40, names:   40 OK
hsv-tile     -> cols:  120, names:  120 OK
color-tile   -> cols:  120, names:  120 OK
he-tile      -> cols:   80, names:   80 OK
locstd       -> cols:  432, names:  432 OK
✅ Generated meta-features with shape: (2088, 1599)


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Saved submission_stacked.csv


In [ ]:
save_folder

'only_lightgbm/version3'

: 

In [ ]:
import numpy as np
import pandas as pd

# 1) 计算平均 feature_importances
importances = np.vstack([
    est.feature_importances_ 
    for est in meta_model.estimators_
]).mean(axis=0)

# 2) 构造 DataFrame 并排序
df_imp = pd.DataFrame({
    "feature": test_name,
    "importance": importances
}).sort_values("importance", ascending=False)


# 方法 A：临时设置最大行数，打印时不截断
pd.set_option('display.max_rows', df_imp.shape[0])
print(df_imp)
# 如果代码运行在脚本里，记得在打印完后恢复默认：
pd.reset_option('display.max_rows')

# 方法 B：直接 to_string()
print(df_imp.to_string(index=False))


                                  feature  importance
543        wavelet_tile_level2_band0_mean  286.028571
635   wavelet_subtile_2_level2_band2_mean  269.485714
651   wavelet_subtile_3_level1_band2_mean  267.257143
615   wavelet_subtile_2_level1_band0_mean  267.200000
791   wavelet_subtile_8_level1_band2_mean  265.400000
707   wavelet_subtile_5_level1_band2_mean  262.914286
579   wavelet_subtile_0_level2_band2_mean  262.771429
747   wavelet_subtile_6_level2_band2_mean  262.228571
627   wavelet_subtile_2_level2_band0_mean  262.085714
803   wavelet_subtile_8_level2_band2_mean  259.000000
679   wavelet_subtile_4_level1_band2_mean  258.942857
821                   sobel-subtile_2_min  258.342857
841                   sobel-subtile_7_min  258.285714
809                        sobel-tile_min  257.885714
719   wavelet_subtile_5_level2_band2_mean  257.771429
787   wavelet_subtile_8_level1_band1_mean  257.428571
813                   sobel-subtile_0_min  256.571429
559   wavelet_subtile_0_leve

In [104]:
import numpy as np
import pandas as pd

# 1) 计算平均 feature_importances
importances = np.vstack([
    est.feature_importances_ 
    for est in meta_model.estimators_
]).mean(axis=0)

# 2) 构造 DataFrame 并排序
df_imp = pd.DataFrame({
    "feature": test_name,
    "importance": importances
}).sort_values("importance", ascending=False)


# 方法 A：临时设置最大行数，打印时不截断
pd.set_option('display.max_rows', df_imp.shape[0])
print(df_imp)
# 如果代码运行在脚本里，记得在打印完后恢复默认：
pd.reset_option('display.max_rows')

# 方法 B：直接 to_string()
print(df_imp.to_string(index=False))


                                  feature  importance
543        wavelet_tile_level2_band0_mean  286.028571
635   wavelet_subtile_2_level2_band2_mean  269.485714
651   wavelet_subtile_3_level1_band2_mean  267.257143
615   wavelet_subtile_2_level1_band0_mean  267.200000
791   wavelet_subtile_8_level1_band2_mean  265.400000
707   wavelet_subtile_5_level1_band2_mean  262.914286
579   wavelet_subtile_0_level2_band2_mean  262.771429
747   wavelet_subtile_6_level2_band2_mean  262.228571
627   wavelet_subtile_2_level2_band0_mean  262.085714
803   wavelet_subtile_8_level2_band2_mean  259.000000
679   wavelet_subtile_4_level1_band2_mean  258.942857
821                   sobel-subtile_2_min  258.342857
841                   sobel-subtile_7_min  258.285714
809                        sobel-tile_min  257.885714
719   wavelet_subtile_5_level2_band2_mean  257.771429
787   wavelet_subtile_8_level1_band1_mean  257.428571
813                   sobel-subtile_0_min  256.571429
559   wavelet_subtile_0_leve

In [105]:
import numpy as np
import pandas as pd

# 1) 计算平均 feature_importances
importances = np.vstack([
    est.feature_importances_ 
    for est in meta_model.estimators_
]).mean(axis=0)

# 2) 构造 DataFrame 并排序
df_imp = pd.DataFrame({
    "feature": test_name,
    "importance": importances
})

# 2) 提取“类别”标签 —— 这里我们以第一个下划线前的字符串当作类别
df_imp['category'] = df_imp['feature'].apply(lambda x: x.split('_')[0])

# 3) 按类别聚合（求和或求平均都可以，下面示例用求和）
cat_imp = df_imp.groupby('category')['importance'] \
            .mean() \
            .sort_values(ascending=False)

print(cat_imp)


# 方法 A：临时设置最大行数，打印时不截断
pd.set_option('display.max_rows', df_imp.shape[0])
print(df_imp)
# 如果代码运行在脚本里，记得在打印完后恢复默认：
pd.reset_option('display.max_rows')

# 方法 B：直接 to_string()
print(df_imp.to_string(index=False))


category
wavelet            178.984048
sobel-tile         176.200000
sobel-subtile      138.058730
locstd             137.253638
wavelet-tile       127.592857
ae                 125.598140
color-tile         113.471429
wavelet-subtile    109.726984
he-tile            107.332143
color-subtile      100.521164
hsv-tile            94.242857
he-subtile          91.593254
hsv-subtile         82.942328
contrast            78.190476
ae-recon-loss       69.000000
latent              68.792857
subtile             66.143122
exsubtiles          56.442857
tile                53.971429
Name: importance, dtype: float64
                                  feature  importance         category
0                    ae-recon-loss_center   69.000000    ae-recon-loss
1                                 ae_emb0  187.714286               ae
2                                 ae_emb1  180.200000               ae
3                                 ae_emb2  187.085714               ae
4                                

In [20]:
import os
import glob
import joblib
import json
import numpy as np
import pandas as pd
import h5py
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneGroupOut
from torch.utils.data import DataLoader

# 1) 预先算好全量 meta‐features
from python_scripts.pretrain_model import PretrainedEncoderRegressor

recon_model = PretrainedEncoderRegressor(
    ae_checkpoint=pretrained_ae_path,
    ae_type=ae_type,
    tile_dim=tile_dim,
    center_dim=center_dim,
    neighbor_dim=neighbor_dim,
    output_dim=C,
    mode='reconstruction'
).to(device)

all_meta = generate_meta_features(
    dataset         = full_dataset,
    model_for_recon = recon_model,
    device          = device,
    ae_type         = ae_type,
)

# 2) 准备 test_meta
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
meta_test = generate_meta_features(
    dataset         = test_dataset,
    model_for_recon = recon_model,
    device          = device,
    ae_type         = ae_type,
)

# 3) 加载每折模型并分别对 val 和 test 做预测
meta_paths = sorted(glob.glob(os.path.join(save_folder, "meta_model_fold*.pkl")))

fold_rmses = []
all_preds  = []

for fold_id, (tr_idx, va_idx) in enumerate(logo.split(
        X = np.zeros(n_samples), y = None, groups = slide_idx
    )):

    model_path = meta_paths[fold_id]
    print(f"\n▶︎ Fold {fold_id} → loading {model_path}")
    meta_model = joblib.load(model_path)

    # 3a) 对测试集做预测
    preds_test = meta_model.predict(meta_test)   # (n_test, C)
    all_preds.append(preds_test)

    # 3b) 对验证集做预测并计算 RMSE
    X_va        = all_meta[va_idx]
    y_va        = y_true[va_idx]
    preds_val   = meta_model.predict(X_va)
    rmse_fold   = np.sqrt(mean_squared_error(y_va, preds_val))
    fold_rmses.append(rmse_fold)
    print(f"   ✔ Fold {fold_id} val RMSE = {rmse_fold:.6f}")

# 保存每折 RMSE
rmses_path = os.path.join(save_folder, "fold_rmses.json")
with open(rmses_path, "w") as f:
    json.dump(fold_rmses, f, indent=2)
print("\n📑 Saved fold RMSEs →", rmses_path)
print("Fold RMSEs:", fold_rmses)

# 4) 根据 RMSE 计算权重（RMSE 越小，权重越大，且和为 1）
fold_rmses = np.array(fold_rmses)
inv         = 1.0 / fold_rmses
weights     = inv / inv.sum()
print("Fold weights:", weights)

# 5) 加权平均 ensemble
all_preds     = np.stack(all_preds, axis=0)  # (K, n_test, C)
preds_weighted= np.tensordot(weights, all_preds, axes=(0,0))  # (n_test, C)

# 6) 输出 submission
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_ids = pd.DataFrame(np.array(f["spots/Test"]["S_7"])).index

df_sub = pd.DataFrame(preds_weighted, columns=[f"C{i+1}" for i in range(C)])
df_sub.insert(0, "ID", test_ids)
out_path = os.path.join(save_folder, "submission_weighted_ensemble.csv")
df_sub.to_csv(out_path, index=False)
print(f"✅ Saved weighted‐ensemble submission → {out_path}")


/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/pretrain_model.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ae.load_state_dict(torch.load(ae_checkpoint, map_

✅ Generated meta-features with shape: (8348, 691)


Computing AE recon loss: 100%|██████████| 33/33 [00:02<00:00, 11.46it/s]


✅ Generated meta-features with shape: (2088, 691)

▶︎ Fold 0 → loading only_lightgbm/version1/meta_model_fold0.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

...

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

   ✔ Fold 4 val RMSE = 8.626677

▶︎ Fold 5 → loading only_lightgbm/version1/meta_model_fold5.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

   ✔ Fold 5 val RMSE = 7.617017

📑 Saved fold RMSEs → only_lightgbm/version1/fold_rmses.json
Fold RMSEs: [np.float64(8.875278692087313), np.float64(7.660174703404531), np.float64(7.493918635679201), np.float64(8.413000279715822), np.float64(8.626677339063225), np.float64(7.617016670460835)]
Fold weights: [0.15170101 0.17576476 0.17966418 0.1600367  0.1560727  0.17676065]
✅ Saved weighted‐ensemble submission → only_lightgbm/version1/submission_weighted_ensemble.csv


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

In [21]:
import glob
import os
import joblib
import numpy as np
import pandas as pd
import h5py
from torch.utils.data import DataLoader

# 1) 读 test spot IDs
with h5py.File("./dataset/elucidata_ai_challenge_data.h5","r") as f:
    test_spot_table = pd.DataFrame(np.array(f["spots/Test"]["S_7"]))
    test_ids = test_spot_table.index

# 2) 准备 DataLoader & AE recon model（用于生成 meta features）
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

recon_model = PretrainedEncoderRegressor(
    ae_checkpoint=pretrained_ae_path,
    ae_type=ae_type,
    tile_dim=tile_dim,
    center_dim=center_dim,
    neighbor_dim=neighbor_dim,
    output_dim=C,
    mode='reconstruction'
).to(device)

# 3) 生成测试集的 meta‐features
meta_test = generate_meta_features(
    dataset         = test_dataset,
    model_for_recon = recon_model,
    device          = device,
    ae_type         = ae_type,
)

# 4) 批量载入各折的 LightGBM meta‐model 并做预测
# save_folder = f"only_lightgbm/{version}"
meta_paths = sorted(glob.glob(os.path.join(save_folder, "meta_model_fold*.pkl")))

all_fold_preds = []
for fold_id, mp in enumerate(meta_paths):
    print(f"▶︎ Predicting fold {fold_id} with {mp}")
    meta_model = joblib.load(mp)
    preds = meta_model.predict(meta_test)      # (n_test, 35)
    all_fold_preds.append(preds)

    # --- 可选：存每折单独的 submission ---
    df_fold = pd.DataFrame(preds, columns=[f"C{i+1}" for i in range(preds.shape[1])])
    df_fold.insert(0, "ID", test_ids)
    path_fold = os.path.join(save_folder, f"submission_lightgbm_fold{fold_id}.csv")
    df_fold.to_csv(path_fold, index=False)
    print(f"✅ Saved fold‐{fold_id} submission → {path_fold}")

all_fold_preds = np.stack(all_fold_preds, axis=0)  # (K, n_test, 35)

# 5) Rank‐average ensemble
#    对每一折的预测做 rank，然后平均
ranks     = all_fold_preds.argsort(axis=2).argsort(axis=2).astype(float)
mean_rank = ranks.mean(axis=0)                    # (n_test, 35)

df_ens = pd.DataFrame(mean_rank, columns=[f"C{i+1}" for i in range(mean_rank.shape[1])])
df_ens.insert(0, "ID", test_ids)
path_ens = os.path.join(save_folder, "submission_lightgbm_rank_ensemble.csv")
df_ens.to_csv(path_ens, index=False)
print(f"✅ Saved rank‐ensemble submission → {path_ens}")

# ——— 如果你想直接做简单平均，也很容易 ———
mean_preds = all_fold_preds.mean(axis=0)
df_avg = pd.DataFrame(mean_preds, columns=[f"C{i+1}" for i in range(mean_preds.shape[1])])
df_avg.insert(0, "ID", test_ids)
path_avg = os.path.join(save_folder, "submission_lightgbm_avg_ensemble.csv")
df_avg.to_csv(path_avg, index=False)
print(f"✅ Saved avg‐ensemble submission → {path_avg}")


/Users/deweywang/Desktop/GitHub/HEVisum/python_scripts/pretrain_model.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ae.load_state_dict(torch.load(ae_checkpoint, map_

✅ Generated meta-features with shape: (2088, 691)
▶︎ Predicting fold 0 with only_lightgbm/version1/meta_model_fold0.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Saved fold‐0 submission → only_lightgbm/version1/submission_lightgbm_fold0.csv
▶︎ Predicting fold 1 with only_lightgbm/version1/meta_model_fold1.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

...

✅ Saved fold‐3 submission → only_lightgbm/version1/submission_lightgbm_fold3.csv
▶︎ Predicting fold 4 with only_lightgbm/version1/meta_model_fold4.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Saved fold‐4 submission → only_lightgbm/version1/submission_lightgbm_fold4.csv
▶︎ Predicting fold 5 with only_lightgbm/version1/meta_model_fold5.pkl


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature na

✅ Saved fold‐5 submission → only_lightgbm/version1/submission_lightgbm_fold5.csv
✅ Saved rank‐ensemble submission → only_lightgbm/version1/submission_lightgbm_rank_ensemble.csv
✅ Saved avg‐ensemble submission → only_lightgbm/version1/submission_lightgbm_avg_ensemble.csv


In [ ]:
import os
import numpy as np
import joblib
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from scipy.stats import rankdata
from python_scripts.import_data import importDataset
from python_scripts.operate_model import predict
from lightgbm import early_stopping, log_evaluation
import h5py
import pandas as pd
from python_scripts.pretrain_model import PretrainedEncoderRegressor
# ---------------- Settings ----------------
trained_oof_model_folder = 'output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/'
n_folds    = len([d for d in os.listdir(trained_oof_model_folder) if d.startswith('fold')])
n_samples  = len(full_dataset)
C          = 35
BATCH_SIZE = 64
start_fold = 0

tile_dim = 128
center_dim = 128
neighbor_dim = 128
fusion_dim = tile_dim + center_dim + neighbor_dim

pretrained_ae_name = 'AE_Center_noaug'
pretrained_ae_path = f"AE_model/128/{pretrained_ae_name}/best.pt"
ae_type = 'center'

# Ground truth label (全 dataset)
y_true = np.vstack([ full_dataset[i]['label'].cpu().numpy() for i in range(n_samples) ])

# Build CV splitter (must match first stage splits)
logo = LeaveOneGroupOut()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

lgb_base = lgb.LGBMRegressor(
    objective='l2',
    metric='rmse',
    learning_rate=0.01,       # 稍微放宽到 0.01，配合更多迭代和 early stopping
    n_estimators=15000,       # 增加上限，early stopping 控制实际轮数
    max_depth=6,              # 控制树深度，减少过拟合风险
    num_leaves=63,            # 2^6-1，与 max_depth 保持一致
    feature_fraction=0.6,     # 每棵树只采样 60% 特征
    bagging_fraction=0.7,     # 每棵树只采样 70% 样本
    bagging_freq=1,           # 每棵树都做 bagging
    min_data_in_leaf=30,      # 叶节点最少要有 30 样本
    reg_alpha=1.0,            # L1 正则化再加强一点
    reg_lambda=1.0,           # L2 正则化再加强一点
    verbosity=-1
)


slide_idx = np.array(grouped_data['slide_idx'])   # shape (N,)


for fold_id, (tr_idx, va_idx) in enumerate(
    logo.split(X=np.zeros(n_samples), y=None, groups=slide_idx)):

    if fold_id > start_fold:
        print(f"⏭️ Skipping fold {fold_id}")
        continue

    print(f"\n🚀 Starting fold {fold_id}...")
    ckpt_path = os.path.join(trained_oof_model_folder, f"fold{fold_id}", "best_model.pt")

    # === Load model and predict OOF ===
    net = VisionMLP_MultiTask(tile_dim=tile_dim, subtile_dim=center_dim, output_dim=C)
    net.load_state_dict(torch.load(ckpt_path, map_location=device))
    net = net.to(device).eval()

    val_ds = Subset(full_dataset, va_idx)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    preds, latents = [], []
    with torch.no_grad():
        for batch in val_loader:
            tiles = batch['tile'].to(device)
            subtiles = batch['subtiles'].to(device)
            center = subtiles[:, 4].contiguous()

            f_c = net.encoder_center(center)
            f_n = net.encoder_subtile(subtiles)
            f_t = net.encoder_tile(tiles)
            fuse = torch.cat([f_c, f_n, f_t], dim=1).contiguous()
            output = net.decoder(fuse)

            preds.append(output.cpu())
            latents.append(fuse.cpu())

    preds = torch.cat(preds, dim=0).numpy()
    latents = torch.cat(latents, dim=0).numpy()

    # === AE model reconstruction loss ===
    recon_model = PretrainedEncoderRegressor(
        ae_checkpoint=pretrained_ae_path,
        ae_type=ae_type,
        tile_dim=tile_dim,
        center_dim=center_dim,
        neighbor_dim=neighbor_dim,
        output_dim=C,
        mode='reconstruction'
    ).to(device)

    meta = generate_meta_features(
        dataset = val_ds,
        oof_preds = preds,
        image_latents = latents,
        model_for_recon = recon_model,
        device = device,
        ae_type = ae_type,
        use_clusters="both"
    )
    
    y_val = y_true[va_idx]   # shape = (len(va_idx), 35)

    # 2) 再對這個 fold 的 meta 做 train/val 切分
    X_train_tab, X_val_tab, y_train_tab, y_val_tab = train_test_split(
        meta, y_val, test_size=0.2, random_state=42
    )

    # 3) train MultiOutputRegressor with early stopping
    meta_model = MultiOutputRegressor(lgb_base)
    meta_model.estimators_ = []

    for i in range(y_train_tab.shape[1]):
        print(f"[fold {fold_id}] training target {i} on meta features …")
        model = lgb.LGBMRegressor(**lgb_base.get_params())
        model.fit(
            X_train_tab, y_train_tab[:, i],
            eval_set=[(X_val_tab, y_val_tab[:, i])],
            callbacks=[
                early_stopping(stopping_rounds=200),
                log_evaluation(period=100)
            ]
        )
        meta_model.estimators_.append(model)

    # 4) 存下這個 fold 的 meta model
    save_path = os.path.join(trained_oof_model_folder, f"meta_model_fold{fold_id}.pkl")
    joblib.dump(meta_model, save_path)
    print(f"✅ Saved fold {fold_id} meta‐model → {save_path}")



🚀 Starting fold 0...


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_58393/2524640228.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(ckpt_path, ma

recon_loss.shape       = (2197,)
latent_stats.shape    = (2197, 4)
stats4.shape          = (2197, 16)
stats20.shape         = (2197, 80)
gt_cluster_stats.shape= (2197, 96)
pred_cluster_stats.shape = (2197, 12)
rgb_center.shape     = (2197, 12)
rgb_all_subs.shape     = (2197, 108)
rgb_except_center_each.shape= (2197, 96)
rgb_except_center.shape= (2197, 12)
rgb_tile.shape         = (2197, 12)
wavelet_feats.shape   = (2197, 14)
hsv_feats.shape       = (2197, 12)
dispersion.shape      = (2197, 1)
oof_preds.shape       = (2197, 35)
image_latents.shape   = (2197, 384)
final features.shape  = (2197, 415)
[fold 0] training target 0 on meta features …
Training until validation scores don't improve for 200 rounds


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


[100]	valid_0's rmse: 6.38718
[200]	valid_0's rmse: 6.23366
[300]	valid_0's rmse: 6.17975
[400]	valid_0's rmse: 6.17796
[500]	valid_0's rmse: 6.17071
[600]	valid_0's rmse: 6.16249
[700]	valid_0's rmse: 6.16253
[800]	valid_0's rmse: 6.16777
Early stopping, best iteration is:
[642]	valid_0's rmse: 6.15622
[fold 0] training target 1 on meta features …
Training until validation scores don't improve for 200 rounds


...

[100]	valid_0's rmse: 7.69747
[200]	valid_0's rmse: 7.6622
[300]	valid_0's rmse: 7.66786
Early stopping, best iteration is:
[199]	valid_0's rmse: 7.66048
[fold 0] training target 33 on meta features …
Training until validation scores don't improve for 200 rounds


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


[100]	valid_0's rmse: 5.68002
[200]	valid_0's rmse: 5.64129
[300]	valid_0's rmse: 5.6362
[400]	valid_0's rmse: 5.63591
[500]	valid_0's rmse: 5.64187
Early stopping, best iteration is:
[360]	valid_0's rmse: 5.63271
[fold 0] training target 34 on meta features …
Training until validation scores don't improve for 200 rounds


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/lightgbm/basic.py:743: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning(


[100]	valid_0's rmse: 3.30132
[200]	valid_0's rmse: 3.19846
[300]	valid_0's rmse: 3.17416
[400]	valid_0's rmse: 3.16159
[500]	valid_0's rmse: 3.15575
[600]	valid_0's rmse: 3.15097
[700]	valid_0's rmse: 3.15044
[800]	valid_0's rmse: 3.15195
[900]	valid_0's rmse: 3.15337
Early stopping, best iteration is:
[761]	valid_0's rmse: 3.14794
✅ Saved fold 0 meta‐model → output_folder/rank-spot/realign/no_pretrain/3_encoder/filtered_directly_rank/k-fold/realign_all/Macenko_masked/meta_model_fold0.pkl
⏭️ Skipping fold 1
⏭️ Skipping fold 2
⏭️ Skipping fold 3
⏭️ Skipping fold 4
⏭️ Skipping fold 5
